<a href="https://colab.research.google.com/github/adityapri/NLP---FastText-Word-Representation/blob/main/FastTEXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tokenization**

In [1]:
import re
from collections import defaultdict
import random
import math

In [2]:
CORPUS_PATH = "/content/wmt-news-crawl-hi.txt"

In [3]:
regx_pattern = r"[\u0900-\u0963\u0966-\u097F]+|\d+"


token_pattern = re.compile(regx_pattern,
                           flags = re.UNICODE)

In [4]:

def tokenize(sent):
  return token_pattern.findall(sent)


def load_corpus(corpus_path):

  sentences = []

  with open(corpus_path, "r",encoding='utf-8') as file:
    for raw_line in file:
      cleaned_line = raw_line.strip()
      if cleaned_line == "":
        continue
      token_list = tokenize(cleaned_line)
      sentences.append(token_list)


  return sentences


In [5]:
 # Load Corpus
sentences = load_corpus(CORPUS_PATH)

print("Total sentences read:", len(sentences))
print("Sample tokens from first sentence:", sentences[:50][:50])

Total sentences read: 6714
Sample tokens from first sentence: [['बरेली', 'जोन', 'के', 'एडीजी', 'रमित', 'शर्मा', 'खुद', 'पूरे', 'मामले', 'की', 'निगरानी', 'कर', 'रहे', 'हैं'], ['बता', 'दें', 'कि', 'सीएल', 'गुप्ता', 'एक्सपोर्ट्स', 'की', 'गिनती', 'मुरादाबाद', 'के', 'दिग्गज', 'पीतल', 'कारोबारियों', 'में', 'होती', 'है'], ['किस', 'देवी', 'की', 'पूजा', 'से', 'क्या', 'लाभ', 'मिलते', 'हैं', 'एक', 'साल', 'में', 'कितनी', 'बार', 'आती', 'हैं', 'नवरात्रियां', 'देवी', 'पूजा', 'के', 'साथ', 'व्रत', 'उपवास', 'क्यों', 'करना', 'चाहिए'], ['थाना', 'रसूलाबाद', 'पुलिस', 'ने', 'नाबालिग', 'किशोरी', 'के', 'साथ', 'रेप', 'करने', 'वाले', 'एक', 'अभियुक्त', 'को', 'गिरफ्तार', 'किया', 'है'], ['हमने', 'विभिन्न', 'मंचों', 'पर', 'यह', 'मुद्दा', 'उठाते', 'हुए', 'इस', 'पर', 'बातचीत', 'का', 'प्रयास', 'किया'], ['उन्होंने', 'कहा', 'कि', 'झारखंड', 'में', 'चुनाव', 'को', 'देखते', 'हुए', 'भाजपा', 'नई', 'नई', 'घोषणाएं', 'कर', 'रही', 'है', 'उनसे', 'पूछा', 'जाना', 'चाहिए', 'कि', 'क्या', 'जो', 'सुविधाएं', 'झारखंड', 'में', 'देने', 'की',

**Vocabulary Construction**

In [6]:
def create_vocabulary(tokenized_corpus):

  word_frequency_map = defaultdict(int)
  token_count = 0

  for sentence in tokenized_corpus:
    for token in sentence:
      word_frequency_map[token] += 1
      token_count += 1

  vocab = sorted(word_frequency_map)
  token_to_index = {tok: i for i, tok in enumerate(vocab)}
  index_to_token = vocab

  return token_to_index, index_to_token, word_frequency_map, token_count

In [7]:
token_index, index_token, freq_dict, token_total = create_vocabulary(sentences)

print("Unique vocabulary size:", len(token_index))
print("Token count in corpus:", token_total)


print("\n50 Most Frequent Words: ")
most_common = sorted(
    freq_dict.items(),
    key=lambda pair : pair[1],
    reverse = True
)[:50]

for token, occurrences in most_common:
    print(f"{token} : {occurrences}")

Unique vocabulary size: 13948
Token count in corpus: 103169

50 Most Frequent Words: 
के : 4268
में : 3148
की : 2681
है : 2597
को : 2007
से : 1915
ने : 1639
का : 1457
और : 1416
पर : 1289
कि : 1071
हैं : 927
भी : 839
कर : 626
नहीं : 622
लिए : 613
एक : 597
इस : 581
किया : 571
गया : 555
था : 489
ही : 479
बाद : 458
हो : 452
साथ : 428
करने : 421
कहा : 421
पुलिस : 404
गई : 389
रहे : 336
दिया : 331
तो : 317
रहा : 307
यह : 306
रही : 304
बताया : 299
थी : 286
हुए : 279
जा : 265
उन्होंने : 260
लेकर : 247
दी : 244
थे : 243
अपने : 233
गए : 229
इसके : 228
सिंह : 222
लेकिन : 217
होने : 201
वह : 196


**Subsampling**

In [8]:
THRESHOLD = 1e-5


def normalize_counts(freq_map, token_count):

    prob_map = {}
    inv_total = 1.0 / token_count

    for term in freq_map:
        prob_map[term] = freq_map[term] * inv_total

    return prob_map


def accept_token(token, prob_map, threshold):

    freq = prob_map[token]

    score = math.sqrt(threshold / freq) + (threshold / freq)
    if score > 1.0:
        score = 1.0

    return random.random() <= score


def apply_subsampling(data, freq_map, token_count):

    probability_table = normalize_counts(freq_map, token_count)

    filtered_data = []

    for sent in data:
        retained = []

        for tok in sent:
            if accept_token(tok, probability_table, THRESHOLD):
                retained.append(tok)

        if len(retained) > 0:
            filtered_data.append(retained)

    return filtered_data


In [9]:
random.seed(42)

reduced_data = apply_subsampling(sentences, freq_dict, token_total)

print("* Corpus Before Subsampling :")
for idx, sent in enumerate(sentences):
    if idx == 20:
        break
    print(sent)

print("\n* Corpus After Subsampling :")
for idx, sent in enumerate(reduced_data):
    if idx == 20:
        break
    print(sent)


* Corpus Before Subsampling :
['बरेली', 'जोन', 'के', 'एडीजी', 'रमित', 'शर्मा', 'खुद', 'पूरे', 'मामले', 'की', 'निगरानी', 'कर', 'रहे', 'हैं']
['बता', 'दें', 'कि', 'सीएल', 'गुप्ता', 'एक्सपोर्ट्स', 'की', 'गिनती', 'मुरादाबाद', 'के', 'दिग्गज', 'पीतल', 'कारोबारियों', 'में', 'होती', 'है']
['किस', 'देवी', 'की', 'पूजा', 'से', 'क्या', 'लाभ', 'मिलते', 'हैं', 'एक', 'साल', 'में', 'कितनी', 'बार', 'आती', 'हैं', 'नवरात्रियां', 'देवी', 'पूजा', 'के', 'साथ', 'व्रत', 'उपवास', 'क्यों', 'करना', 'चाहिए']
['थाना', 'रसूलाबाद', 'पुलिस', 'ने', 'नाबालिग', 'किशोरी', 'के', 'साथ', 'रेप', 'करने', 'वाले', 'एक', 'अभियुक्त', 'को', 'गिरफ्तार', 'किया', 'है']
['हमने', 'विभिन्न', 'मंचों', 'पर', 'यह', 'मुद्दा', 'उठाते', 'हुए', 'इस', 'पर', 'बातचीत', 'का', 'प्रयास', 'किया']
['उन्होंने', 'कहा', 'कि', 'झारखंड', 'में', 'चुनाव', 'को', 'देखते', 'हुए', 'भाजपा', 'नई', 'नई', 'घोषणाएं', 'कर', 'रही', 'है', 'उनसे', 'पूछा', 'जाना', 'चाहिए', 'कि', 'क्या', 'जो', 'सुविधाएं', 'झारखंड', 'में', 'देने', 'की', 'बात', 'भाजपा', 'वाले', 'कर', 'रहे', 

**N-Gram Generation**

In [13]:
def extract_char_ngrams(token, low=3, high=5):

    results = []


    marked = "<" + token + ">"
    size = len(marked)


    for n in range(low, high + 1):
        for start in range(size - n + 1):
            gram = marked[start:start + n]
            results.append(gram)


    results.append(marked)

    return results


In [14]:
def prepare_ngram_resources(corpus, vocab_map, low=3, high=5):


    token_to_grams = {}
    unique_grams = set()


    for token in vocab_map:
        grams = extract_char_ngrams(token, low, high)
        token_to_grams[token] = grams

        for g in grams:
            unique_grams.add(g)


    index_to_gram = list(sorted(unique_grams))
    gram_to_index = {}

    for idx, gram in enumerate(index_to_gram):
        gram_to_index[gram] = idx

    return token_to_grams, gram_to_index, index_to_gram


In [16]:
# Build n-gram structures
token_to_grams, gram_to_index, index_to_gram = prepare_ngram_resources(
    corpus=reduced_data,
    vocab_map=token_index
)

print("Vocabulary size:", len(token_to_grams))
print("Total n-grams:", len(gram_to_index))

# Examine a Hindi example
sample = "भारत"
print(f"\nCharacter n-grams for '{sample}':")
print(token_to_grams[sample])


# print(gram_to_index)


Vocabulary size: 13948
Total n-grams: 93506

Character n-grams for 'भारत':
['<भा', 'भार', 'ारत', 'रत>', '<भार', 'भारत', 'ारत>', '<भारत', 'भारत>', '<भारत>']
